# Selección del tamaño de muestra

    - Un tamaño de muestra -n- muy pequeño implica inferencias inútiles o intervalos de confianza muy grandas, mientras que un tamaño de muestra grande implica que existan costos elevados o no se pueda garantizar la calidad de la información
    
    - Para las implicaciones siguientes se supone la normalidad en el estimador.

### 1] Definir una población
    - Gracias al TLC, sin importar la distribución de la VA, es posible realizar las estimaciones de las medias, totales y proporciones

In [80]:
import pandas as pd
import numpy as np
from scipy.stats import norm


# Define a population --- --- --- --- --- --- --- --- --- --- --- --- --- --- --- --- --- --- --- --- --- 

# Settings
confidence = 0.99
N = 1000
n_pilot = 50
P = 0.35

# Define columns to be included
score = np.random.normal(loc=6.5, scale=1.5, size=N)
money = np.random.randint(low=200, high=1200, size=N)
is_male = np.where(np.random.uniform(size=N)<P, 1, 0)

# Put together data into a df
df_population = pd.DataFrame(np.vstack([score, money, is_male]).T, columns=["score", "money", "is_male"])



# Select a random sample --- --- --- --- --- --- --- --- --- --- --- --- --- --- --- --- --- --- --- --- --- 

# Create auxiliary column to randomize the sample
df_population["mask"] = np.random.uniform(size=N)

# Sort and select elements randomly
df_pilot = df_population.sort_values(by=["mask"])[:n_pilot]

# Drop auxiliary column
df_population.drop(["mask"], axis=1, inplace=True)

### 1] Promedio - Calcular tamaño de muestra para estimación

    - Se selecciona una precisión (d) y un nivel confianza (c) (1-alpha)
    
    - Se realiza una prueba piloto para calcular:
        - mean = sum(y_i) / n
        - var = sum( (y_i-mean)^2 ) / (n-1)
        - std = var^0.5
    
    - Cuando N es grande:
        n0 = ( std * z(1-alpha/2) / d )^2
            donde: 
            z(1-alpha/2) es la inversa de una función normal estándar
            d = error absoluto (en unidades de la media)
        
            
    - Cuando N es pequeña
        n = n0 / (1 + n0/N)
            
    - Existen fórmulas específicas que se pueden utilizar para expresar el error de manera relativa (x% mean) en vez de errores absolutos (en unidades de mean)
    
    - Nótese que se necesita conocer S^2 para conocer el tamaño de muestra. Lo que se sugiere es hacer una prueba piloto o de encuestas anteriores

In [81]:
# Calculation is done using "score" variable

# Settings
d = 0.50
print("Estimate mean of score with "+str(int(confidence*100))+"% of confidence")
print("Expected error: "+str(d))

# Estimate pilot variance
mean = df_pilot["score"].mean()
var = df_pilot["score"].var(ddof=1)
std = var**0.5
print("mean: "+str(round(mean,2))+" - stdev: "+str(round(std,2)))

# Calculate sample size
n0 = (std * norm.ppf(0.5+confidence/2) / d)**2
n = np.ceil(n0 / (1+n0/N))
print("\nSample size: "+str(int(n)))

Estimate mean of score with 99% of confidence
Expected error: 0.5
mean: 6.55 - stdev: 1.51

Sample size: 58


### 2] Total - Calcular tamaño de muestra para estimación

    - Se realiza una prueba piloto para calcular la varianza de las observaciones
        - total = N * mean = (N/n) * sum(y_i)
        
    - Cuando N es grande:
        n0 = ( N * std * z(1-alpha/2) / d )^2
            donde: 
            z(1-alpha/2) es la inversa de una función normal estándar
            d = error absoluto (en unidades de la media)
        
            
    - Cuando N es pequeña
        n = n0 / (1 + n0/N)

In [82]:
# Analysis is done using "money" variable

# Settings
d = 50000
print("Estimate total of money with "+str(int(confidence*100))+"% of confidence")
print("Expected error: "+str(d))

# Estimate pilot variance
total = df_pilot["money"].mean()
var = df_pilot["money"].var(ddof=1)
std = var**0.5
print("total: "+str(round(total*N,2))+" - stdev: "+str(round(std,2)))

# Calculate sample size
n0 = (N * std * norm.ppf(0.5+confidence/2) / d)**2
n = np.ceil(n0 / (1+n0/N))
print("\nSample size: "+str(int(n)))

Estimate total of money with 99% of confidence
Expected error: 50000
total: 665480.0 - stdev: 270.72

Sample size: 163


### 3] Proporción - Calcular tamaño de muestra para estimación

    - Se realiza una prueba piloto para calcular la varianza de las observaciones
        - total = N * mean = (N/n) * sum(y_i)
        
    - Cuano N es muy grande:
        n0 = ( z(1-alpha/2) * P * (1-P) ) / d^2
        donde:
            z(1-alpha/2) es la inversa de una función normal estándar
            d = error absoluto (en unidades de la media) 
            P = proporción
    
    - Cuando N es pequeña:
            n = n0 / (1 + n0/N)

In [91]:
# Analysis is done using "is_male" variable

# Settings
d = 0.15
print("Estimate total of is male with "+str(int(confidence*100))+"% of confidence")
print("Expected error: "+str(d))

# Estimate pilot proportion
prop = df_pilot["is_male"].sum() / n_pilot
print("proportion: "+str(round(prop,2)))

# Calculate sample size
n0 = (norm.ppf(0.5+confidence/2)**2 * prop * (1-prop)) / d**2
n = np.ceil(n0 / (1+n0/N))
print("\nSample size: "+str(int(n)))

Estimate total of is male with 99% of confidence
Expected error: 0.15
proportion: 0.32

Sample size: 61
